In [35]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sktime.datasets import load_macroeconomic
from sktime.forecasting.compose import make_reduction
from sktime.transformations.series.summarize import WindowSummarizer
from sklearn.ensemble import RandomForestRegressor

In [36]:
pd.set_option("display.max_columns", None)

In [37]:
data = load_macroeconomic()
y = data["unemp"]
X = data.drop(columns=["unemp"])

print(X.shape)

pd.concat([X, y], axis=1).head()

(203, 11)


,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,pop,infl,realint,unemp
Period,,,,,,,,,,,,
1959Q1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,177.146,0.00,0.00,5.8
1959Q2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,177.830,2.34,0.74,5.1
1959Q3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,178.657,2.74,1.09,5.3
1959Q4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,179.386,0.27,4.06,5.6
1960Q1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,180.007,2.31,1.19,5.2


In [38]:
10 * 1 + 11 * 10

120

In [41]:
regressor = LinearRegression(fit_intercept=False)

kwargs = {
    "lag_feature": {
        "lag": [1],
        # "mean": [[1, 3], [3, 6]],
        # "std": [[1, 4]]
    }
}

forecaster1 = make_reduction(
    RandomForestRegressor(random_state=1),
    window_length=None,
    transformers=[WindowSummarizer(**kwargs, n_jobs=1, truncate="bfill")],
    strategy="recursive",
    pooling="local",
)

forecaster1.fit(y=y, X=X, fh=[1, 2, 3, 4, 5])

coeff = forecaster1.get_fitted_params()["estimator__coef"]
print(
    f'Number of features with pooling local: {forecaster1.get_fitted_params()["estimator__n_features_in"]}'
)
print(f"Number of features with pooling local: {len(coeff)}")

ValueError: Transformers currently cannot be providedfor models that run locally

In [20]:
forecaster1.predict(fh=[1, 2, 3, 4, 5])

2009Q4    9.749450
2010Q1    9.696733
2010Q2    9.435983
2010Q3    9.235430
2010Q4    9.046891
Freq: Q-DEC, Name: unemp, dtype: float64

In [42]:
kwargs = {
    "lag_feature": {
        "lag": [1],
        # "mean": [[1, 3], [3, 6]],
        # "std": [[1, 4]]
    }
}

forecaster2 = make_reduction(
    regressor,
    scitype="tabular-regressor",
    window_length=None,
    strategy="recursive",
    transformers=[WindowSummarizer(**kwargs, n_jobs=1, truncate="bfill")],
    pooling="global",
)
forecaster2.fit(X=X, y=y, fh=[1, 2, 3, 4, 5])
print(
    f'Number of features with pooling global: {forecaster2.get_fitted_params()["estimator__n_features_in"]}'
)

Number of features with pooling global: 12
